# Rankine-Hugoniot, choc attaché et réflexion

On vous propose de mettre en place ...

`Python` sera utilisé ici comme `matlab`. Des fonctionnalités supplémentaires peuvent être ajoutées par l'import de modules, standards à une distribution (comme `math`, `numpy`) ou personnalisés comme ci-dessous. Des fonctionnalités d'édition sont propres à [`Ipython/Notebook`](#ipython).

In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt
from aerokit.common import defaultgas
from aerokit.aero import degree    as deg # import trigo functions with degree unit support
from aerokit.aero import ShockWave as sw  # import functions for shockwave computation
import aerokit.aero.plot.shockpolar as swplt
import ipywidgets as pyw
%matplotlib inline

On définit tout d'abord les paramètres de ce cas. Ils sont définis comme des variables globales, que l'on peut utiliser directement dans les fonctions (comme $\gamma$ par exemple).

In [3]:
# definition of problem parameters
gam  = 1.4 ; defaultgas.set_gamma(gam)

# function to plot the geometry
#
def plot_geom(dev, xneg=-.5, length=2., zoom=1, ax=plt):
    #fig = plt.figure(figsize=(14*zoom,8*zoom))
    #plt.axis([xneg, length])
    ax.set(aspect="equal", xlim=[xneg, length], ylim=[-.1, 1.1])
    ax.plot([xneg, length],    [1, 1],                      color="black", linewidth=2)
    ax.plot([xneg, 0, length], [0, 0, length*deg.tan(dev)], color="black", linewidth=2)
#
# test de la fonction de tracé   

## Premier choc attaché

In [4]:
w_M1 = pyw.Text(description="$M_1$", disabled=True)
w_M2 = pyw.Text(description="$M_2$", disabled=True)
w_res = pyw.VBox([w_M1, w_M2])
                  
def plot_all(M0, wdev):
    devmax   = sw.dev_Max(M0)
    devsonic = sw.dev_Sonic(M0)
    sig1 = sw.sigma_Mach_deflection(M0, wdev)
    Mn0  = M0*deg.sin(sig1)
    p1p0 = sw.Ps_ratio(Mn0)
    Mn1  = sw.downstream_Mn(Mn0)
    M1   = Mn1/deg.sin(sig1-wdev)
    sig2 = sw.sigma_Mach_deflection(M1, wdev)
    Mn1  = M1*deg.sin(sig2)
    p2p1 = sw.Ps_ratio(Mn1)
    Mn2  = sw.downstream_Mn(Mn1)
    M2   = Mn2/deg.sin(sig2-wdev)
    p2p0 = p2p1*p1p0
    yend=.8 # parameter to draw reflected shock
    #
    w_M1.value = "{:5.3f}".format(M1)
    w_M2.value = "{:5.3f}".format(M2)
    #
    #fig = plt.figure(figsize=(16,8))
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,6), gridspec_kw={'width_ratios': [2, 1]})
    fig.tight_layout()
    plot_geom(dev=wdev, zoom=.8, ax=ax1)
    xup  = 1./deg.tan(sig1)                     # impact of first shock on top wall
    xbot = xup + (1.-yend)/deg.tan(sig2-wdev)   # abscissa of reflected shock at yend
    ax1.plot([0, xup, xbot], 
             [0,   1, yend], 'red', linewidth=2)
    #fig=swplt.figure_theta_pressure(figsize=(14,8))
    swplt.set_grid(ax2)
    #fig.suptitle('Polar of Shock-Waves, $\gamma = %.1f$'%gam, fontsize=12, y=0.93)
    ax2.set_xlabel('flow angle', fontsize=10)
    ax2.set_ylabel('normalized static pressure', fontsize=10)
    if p2p0 > 20.: ax2.set_yscale('log') # default is logarithmic
    #
    # plot polar curves
    swplt.plot_theta_pressure(M0, devmax=True, sonic=True, curve='right', ax=ax2)
    swplt.plot_theta_pressure(M1, thet_init=wdev, p_init=p1p0, curve='left', color='red', ax=ax2)

    # plot symbols for flow regions
    ax2.plot(0,    1.,   'bo') ; ax2.annotate("0", (0,    1.),   xytext=(0,5), textcoords='offset points')
    ax2.plot(wdev, p1p0, 'bo') ; ax2.annotate("1", (wdev, p1p0), xytext=(5,-10), textcoords='offset points')
    ax2.plot(0.,   p2p0, 'go') ; ax2.annotate("2", (0,    p2p0), xytext=(5,5), textcoords='offset points')

# Représentation dans la polaire

In [5]:
w_M0  = pyw.FloatSlider(value=3, min=1, max=10, step=.05, description="$M_0$")
w_dev = pyw.FloatSlider(value=10., min=0., max=40, step=.2, description="$\\theta_w$")

w_control = pyw.HBox([pyw.VBox([w_M0, w_dev]), w_res])

out = pyw.interactive(plot_all, M0=w_M0, wdev=w_dev)
out.children[-1].layout.height = '400px'

w_T1 = pyw.VBox([w_control, out.children[-1]])
w_T2 = pyw.VBox([
        pyw.FloatRangeSlider(value=[1., 10.], min=0, max=100.0, step=1,
            description='$M_0$ range', readout=True, readout_format='.1f'),
        pyw.FloatRangeSlider(value=[0., 30.], min=0, max=60.0, step=1,
            description='$\\theta_w$ range', readout=True, readout_format='.1f')  ])
output = pyw.Tab([w_T1, w_T2])
output.set_title(0, 'Shock reflexion')
output.set_title(1, 'Settings')
output

---

<a id="ipython"></a>
## Ipython et notebook : usage

* le notebook utilise la langage de base python en version améliorée, Ipython, qui permet la complétion des noms (variables, fonctions, modules) avec la touche tabulation
* toutes les cellules peuvent être modifiées par un double-clic et sont réinterprêtées avec `shift-entrée`
* l'ensemble de la feuille peut être exécutée avec le menu `Cell/run all cells`
* **n'oubliez pas de sauvegarder régulièrement votre feuille** (bouton _enregistrer_)


In [6]:
from IPython.core.display import HTML ; HTML(open("../custom.css", "r").read()) # notebook style